# Case study 1: The dataset

## The used car data set


The dataset was found on Kaggle [here](https://www.kaggle.com/adityadesai13/used-car-dataset-ford-and-mercedes).

Here is the text describing the dataset:

> **Updated**
>
> Scraped data of used cars listings. 100,000 listings, which have been separated into files corresponding to each car manufacturer. I collected the data to make a tool to predict how much my friend should sell his old car for compared to other stuff on the market, and then just extended the data set. Then made a more general car value regression model.
>
> **Previous version**
>
> Picked two fairly common cars on the British market for analysis (Ford Focus and Mercedes C Class). The hope is to find info such as: when is the ideal time to sell certain cars (i.e. at what age and mileage are there significant drops in resale value). Also can make comparisons between the two, and make a classifier for a ford or Mercedes car. Can easily add more makes and models, so comment for any request e.g. if you want a big data set of all Mercedes makes and models.
>
> **Content**
>
> The cleaned data set contains information of price, transmission, mileage, fuel type, road tax, miles per gallon (mpg), and engine size. I've removed duplicate listings and cleaned the columns, but have included a notebook showing the process and the original data for anyone who wants to check/improve my work.
Inspiration
>
> It'd be cool to have some insights and visualizations of the data. Also, am open to ideas on how to expand the data set.

Here the data collection was performed by someone else and we directly use the result of his work. So our first job will be to understand how the dataset is structured.

## Discovering the dataset

The dataset is composed of several files corresponding to some brands and even some models (check the "previous version" section). Note that some files are duplicated in the sense that there is a "clean" and an "unclean" version.

### Open the data files

The dataset is located in the following directory:

In [ ]:
%load_ext autoreload
%autoreload 2

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


DATA_DIR = "../../data/car_price_dataset"

**Exercise**:
1. List the files in the data set directory
2. Retrieve the list `csv_files` of the "clean" files only, sorted alphabetically

In [ ]:
import os

csv_files = os.listdir(DATA_DIR)
csv_files

In [ ]:
csv_files = sorted(
    [
        os.path.join(DATA_DIR, fname)
        for fname in os.listdir(DATA_DIR)
        if not fname.startswith("unclean")
    ]
)

## or

csv_files = [
    os.path.join(DATA_DIR, fname)
    for fname in os.listdir(DATA_DIR)
    if not "unclean" in fname
]
csv_files.sort()
csv_files

**Note**: When using `listdir`, the file names are returned as provided by the operating system. **There is no guarantee that the order is reproducible**. This means you should usually **sort the result**.

Let's open the first file:

In [ ]:
import pandas as pd

df = pd.read_csv(csv_files[0])
print(df.info())
df.head()

**Exercise**:
1. Write a function that:
    1. reads a csv file
    2. adds a column called `brand` filled from the filename (without extension)
    3. returns the data frame

```python
def open_file(path):
    """Open a file and add the brand column

    Parameters:
    path: str
        Path of the file to open
    """
    pass
```
2. Read and concatenate all the files in a unique data frame `df_cars`


In [ ]:
def open_file(path):
    """Open a file and add the brand column

    Parameters:
    path: str
        Path of the file to open
    """
    df = pd.read_csv(path)
    brand = os.path.splitext(os.path.basename(path))[0]
    df["brand"] = brand
    return df.rename(columns={"tax(£)": "tax"})


df_cars = pd.concat(open_file(fname) for fname in csv_files)
df_cars

The `mll1.car_pricing` module contains a version of this function for further use. Let's use it:

In [ ]:
import sys

sys.path.append("../python")
from mll1 import car_pricing

df_cars = pd.concat(car_pricing.open_file(fname) for fname in csv_files).reset_index(
    drop=True
)
df_cars

In [ ]:
df_cars.info()

**Exercise**: Enumerate some possible problems we could encounter in the table value.

1. Missing values
2. Abnormal values
3. Synonyms in categorical variables
4. Miswritten category names
5. Character encoding
6. Duplicated rows

### Sanity checks on string values

Let's first retrieve string values columns. These are typed as `object` in the `dtype` attribute of the dataframe:

In [ ]:
cat_columns = [c for c in df_cars.columns if df_cars.dtypes[c] == "object"]
cat_columns

One common source of troubles comes from the fact that some string values may exhibit leading or trailing white spaces. It can be difficult to spot because the spaces are not easily seen when displaying the data content.

In the code below, we check whether the column content is modified by stripping those white spaces:

In [ ]:
print("Check if there are leading or trailing spaces:")
for c in cat_columns:
    print(f"{c:15}: {(df_cars[c] != df_cars[c].str.strip()).any()}")

We can see that values in the `model` column exhibit the problem. It might be a good idea to check the column names too.

A second source of trouble can come from **case variability** (upper/lower) in the values of categorical variables.

**Exercise**: Check for case variability by performing the following steps:

1. Count the occurencies of categories in each column
2. Do the same thing after:
    1. Stripping the white spaces
    2. Normalizing the case (using the `title()` method)
3. Check that the number of categories is the same
4. Check that the corresponding counts are the same
5. Conclude


In [ ]:
print("Check if there is some case variability:")
for c in cat_columns:
    df_counts_orig = df_cars[c].value_counts()
    df_counts_modified = df_cars[c].str.strip().str.title().value_counts()
    print(
        f"{c:15}: "
        f"Same number of categories: {len(df_counts_orig) == len(df_counts_modified)}. "
        f"Same counts: {(df_counts_modified.values == df_counts_orig.values).all()}"
    )

### Univariate analysis

Let's analyze the distribution of model values. The model value determines completely the brand value since different brands are not supposed to use the same model names. So let's display model distributions by brand:

In [ ]:
import seaborn as sns

_ = fig = sns.catplot(
    data=df_cars.assign(model=df_cars.model.apply(repr)),
    y="model",
    kind="count",
    col="brand",
    col_wrap=3,
    sharey=False,  # Allows to have different models for each facet
    color="tab:blue",
)

Note : We used the `repr` function to check that there are no additional white spaces in the model names.

We can remark that:

* many models have very few counts,
* there are two "brands" which are actually models: 'Focus' and 'C Class'.

Let's have a look at transmission:

In [ ]:
transm_counts = df_cars.transmission.value_counts()
_ = transm_counts.plot.pie(autopct="%0.2f%%", explode=[0.1] * len(transm_counts))

We can see that there are 3 main categories plus an 'Other' category...

Now let's have a look at the fuel type:

In [ ]:
import matplotlib.pyplot as plt

fuel_counts = df_cars.fuelType.value_counts()
fuel_counts.plot.bar()
ymin, ymax = plt.ylim()
for i, val in enumerate(fuel_counts):
    plt.text(
        i,
        val + ymax * 0.02,
        f"{val}\n({val / fuel_counts.sum():0.2%})",
        horizontalalignment="center",
    )
plt.ylim(0, ymax * 1.1)
plt.xlabel("fuelType")
plt.ylabel("Frequency")
_ = plt.xticks(rotation=45)

We can see that `Electric` fuel type has only very few examples. This means we have probably not enough examples to learn something relevant. There are at least 2 possibilities:

* ignore those points
* fuse them with the 'Other'

Let's have a look at the price distribution for those two categories:

In [ ]:
_ = sns.histplot(
    data=df_cars.query("fuelType == 'Other' or fuelType == 'Electric'"),
    x="price",
    hue="fuelType",
    stat="density",
    common_norm=False,
)

It seems reasonable to **fuse the two classes**.

Now let's have a look at the `year` distribution:

In [ ]:
df_cars.year.value_counts().sort_index().plot.bar(log=True)
plt.xlabel("year")
_ = plt.ylabel("Frequency (log)")

There are several interesting points:

* There is an exponentially decreasing tail with the car age (is it expected ?)
* There is 1 car from 2060, it's probably a mistake. It will be reasonable to ignore this point.
* Note the few cars at **1970**.

The `1970-01-01` date has a special meaning. It is called the **Unix epoch**, and is used as a reference date to encode timestamps (which are often stored as integer values representing the number of seconds since the epoch). A null timestamp would result in that particular date (and midnight as the time).

The data points with those dates are then most probably the result of a missing date information.

In [ ]:
df_cars.query("year == 1970 or year == 2060").shape[0]

Those 3 points can probably be safely ignored.

Now let's have a look at the `mileage` distribution:

In [ ]:
df_cars.mileage.plot.hist(bins=100, log=True)
_ = plt.xlabel("mileage")

We have a clear exponential tail. Besides, there are a few point with a very high mileage value. Are they real or mistakes? Should we keep them?

The next variable we will analyze is the car consumption or `mpg` (for "miles per gallon") variable:

In [ ]:
df_cars.mpg.plot.hist(bins=100, figsize=(12, 4))
_ = plt.xlabel("mpg")

We see a very peaked distribution, with a couple very low values and a few very high ones.

**Exercise**:

* What can be the reason for very high values ?
* What can be the reason for very low values ?

**High values** can be due to:

* Mistakes
* Hybrid / Electric cars
* Uncommon fuel types ?

**Low values** can be due to:

* Mistakes
* Very old cars
* Unit errors (american gallons vs imperial gallons, mpg vs km/l)

Let's check what fuel types correspond to high values:

In [ ]:
_ = df_cars.query("mpg > 130").fuelType.value_counts().plot.bar()

As expected most cars with very high mpg values are in the "Hybrid", "Other" and "Electric". There are a few cars in "Diesel" and "Petrol" categories too. Those are probably errors, we might want to ignore them. (What is the meaning of mpg for electric cars?)

What about low values?

In [ ]:
df_cars.query("mpg < 15").mpg.plot.hist(bins=100)
_ = plt.xlabel("mpg")

In [ ]:
df_cars.query("mpg < 15").year.value_counts().sort_index().plot.bar(log=True)
plt.xlabel("year")
_ = plt.ylabel("Frequency (log)")

The affected entries all correspond to recent cars. Could these be electric (or hybrid models), where `mpg` might have a different meaning ?

In [ ]:
df_cars.query("mpg < 15").fuelType.value_counts().sort_index().plot.bar(log=True)
plt.xlabel("Fuel Type")
_ = plt.ylabel("Frequency (log)")

This is not conclusive either. Since they represent very few samples, it might be a good idea to remove them.

Let's have a look at the tax information:

In [ ]:
df_cars.tax.plot.hist(bins=50, log=True)
_ = plt.xlabel("tax")

There are clearly 3 groups.

How about the engine size ?

In [ ]:
plt.figure(figsize=(12, 5))
df_cars.engineSize.value_counts().sort_index().plot.bar(log=True)
_ = plt.xlabel("engineSize")

We can see nearly 300 entries at 0. Are those missing values ?

In [ ]:
df_cars.query("engineSize == 0.0").fuelType.value_counts().plot.bar()
plt.xlabel("fuelType")
_ = plt.ylabel("Frequency")

Finally looking at the sale price:

In [ ]:
df_cars.price.plot.hist(bins=100, log=True)
_ = plt.xlabel("price")

We clearly have an exponential tail here too. With a few samples at very high prices. The choice to keep or remove the extreme values is difficult and should be motivated by the project goal. Are they real prices (maybe from collection cars) or mistakes? If they are real, does our friend often deal with collection cars ?


### Data cleaning

Now that we have a basic understanding of our dataset, it would be a good time to perform some data cleaning before going further into the analysis.

We will now put in place different functions to clean the dataset. These will:

* apply some normalization to categorical string values (remove spaces, ensure consistent capitals, etc.);
* fuse some categories;
* replace some categorical values;
* add some quality indicators.

Instead of removing right now the data samples with poor quality or missing values, we will add some quality and missing value indicators to the dataframe.  This allows us to postpone the decisions related to the way those values will be processed.

This allows us to make a distinction between two potentially overlapping stages:

* **data cleaning / preparation**: which builds the data set in a consistent way to ease further analyses ;
* **data preprocessing**: which implies decisions depending on the modeling strategy.

#### String normalization

In [ ]:
def normalize_string_values(df, columns):
    """Normalize string values

    Normalize string value by removing trailing and leading white spaces
    and applying a `title` transformation.

    Parameters
    ----------
    df: pandas.DataFrame
        The input dataframe

    columns: seq of str
        The columns to treat

    Returns
    -------
    df_norm: pandas.DataFrame
        The treated data frame
    """
    df_norm = df.copy()
    for c in columns:
        df_norm[c] = df[c].str.strip().str.title()
    return df_norm

#### Fusing categories

In [ ]:
def fuse_string_values(df, column, values, fused_value):
    """Fuse string values

    Replace several string values by a single value. Other values will
    remain unchanged.

    Parameters
    ----------
    df: pandas.DataFrame
        The input data frame.

    column: str
        The column to treat.

    values: seq of str
        The values to fuse.

    fused_value: str
        The value to use to replace fused values

    Returns
    -------
    df_fused: pandas.DataFrame
        The treated data frame
    """
    return df.assign(
        **{column: df[column].apply(lambda x: fused_value if x in values else x)}
    )

#### Replace some values

In [ ]:
def replace_values(df, column, value_map):
    """Replace values

    Parameters
    ----------
    df: pandas.DataFrame
        The input data frame

    column: str
        The name of the column to treat.

    value_map: dict[str, str]
        The values to replace as (original_value, replacement_value)
        pairs.

    Returns
    -------
    df_repl: pandas.DataFrame
        The treated data frame
    """
    return df.assign(**{column: df[column].apply(lambda x: value_map.get(x, x))})

#### Add some quality indicators

In [ ]:
def add_quality_column(df, funcs, suffix="_quality"):
    """Add columns to assess column quality.

    Columns are added to assess original column qualities. The
    new columns have the same name as the original colum with an
    added suffix. The quality assessment is done by providing functions
    for each column.

    Parameters
    ----------
    df: pandas.DataFrame
        The input data frame

    funcs: dict[str, callable]
        A dictionary with (column_name, function) pairs.

    Returns
    -------
    df_qual: pandas.DataFrame
        The data frame with additional quality columns.
    """
    return df.assign(
        **{column + suffix: func(df[column]) for column, func in funcs.items()}
    )

#### Putting it all together

We also build a function to load the dataset:

In [ ]:
def load_dataset(data_dir):
    """Load the used cars dataset"""
    csv_files = [
        os.path.join(data_dir, fname)
        for fname in os.listdir(data_dir)
        if "unclean" not in fname and fname.endswith(".csv")
    ]
    return pd.concat(open_file(fname) for fname in sorted(csv_files)).reset_index(
        drop=True
    )

**Note** the use of the `sorted` function.

And put everything together into a preparation function:

In [ ]:
def prepare_dataset(df):
    """Prepare the data set"""
    cat_cols = [c for c in df.columns if df.dtypes[c] == "object"]
    df = normalize_string_values(df, cat_cols)
    df = fuse_string_values(df, "fuelType", ["Other", "Electric"], "Electric + Other")
    df = replace_values(df, "brand", {"Focus": "Ford", "Cclass": "Merc"})

    quality_funcs = {
        "year": lambda x: np.where(np.in1d(x, [2060, 1970]), "ko", "ok"),
        "engineSize": lambda x: np.where(x == 0.0, "ko", "ok"),
    }

    df = add_quality_column(df, quality_funcs)
    # For MPG we need conditions on 2 variables
    df["mpg_quality"] = np.where(
        (df.mpg < 15) | ((df.mpg > 130) & (df.fuelType.isin(["Petrol", "Diesel"]))),
        "ko",
        "ok",
    )

    df["tax_mpg_missing"] = df["tax"].isnull()

    return df

In [ ]:
df_cars = load_dataset("../../data/car_price_dataset/")
df_cars_prep = prepare_dataset(df_cars)

In [ ]:
df_cars_prep.info()

To be able to reuse them later in other notebooks we have put all those functions in a `car_pricing` python module:

In [ ]:
from mll1 import car_pricing

df_cars = car_pricing.load_dataset("../../data/car_price_dataset/")
df_cars_prep = car_pricing.prepare_dataset(df_cars)
df_cars_prep

### Univariate density estimation

In the above examples, we have visualized the distribution of continuous variables using histograms. Histograms consist in discretizing the continuous variable into bins and counting the number of points falling in the bins. In some cases, we might be interested in having a **smooth** estimation of the density over the full definition domain.

**Density estimation** aims at computing an **approximate probability density function** (PDF) from the data samples. Approximated PDFs can be used for :

* visualization: they are more convenient than histograms when several distributions need to be compared;
* as an estimation of the true PDF in algorithms that rely on the knowledge of the PDF.

Density estimation is an **unsupervised learning** task.

#### Parametric density estimation

Parametric density estimation consists in **fitting** the parameters of a given distribution to the data set.

The choice of the distribution to fit is of course important. There are two main cases:

* If the underlying distribution of the model is known (based on some physical model for instance) then we of course select the known distribution;
* If the underlying distribution of the model is not known we will select a distribution which have the appropriate properties (definition domain, symmetry, long tail, etc.)

Let's have a look at the price distribution:

In [ ]:
x = df_cars.price
x.plot.hist(bins=100, density=True)
_ = plt.xlabel("price")

We can use a log representation to see the properties of the tail:

In [ ]:
x = df_cars.price
x.plot.hist(bins=100, density=True, log=True)
_ = plt.xlabel("price")

**Note**: We use the `density=True` parameter. With this option we have the sum of the bin values multiplied by the bin width which sums to one. The above figure can then be seen as a **piecewise constant approximation** of the distribution.

The distribution has the following notable properties:

* It is non-negative
* It is skewed on the right
* It has a long tail

Among the distributions that could fulfill the above properties we could try:

* The log-normal distribution
* The inverse Gaussian distribution
* The Gamma distribution
* The inverted Gamma distribution

You can refer to this [Wikipedia page](https://en.wikipedia.org/wiki/List_of_probability_distributions) for a list of distributions.

Let's try and fit those distributions on the price data. We can use the implementations givens by the `scipy.stats` package:

In [ ]:
from scipy import stats

x.plot.hist(bins=100, density=True, log=True, alpha=0.5)
xmin, xmax = plt.xlim()
ymin, ymax = plt.ylim()

xx = np.linspace(1, xmax, 501)

distributions = {}
for name in ["invgauss", "gamma", "invgamma", "lognorm"]:
    dist = getattr(stats, name)  # Get the function using its name
    params = dist.fit(x)  # Fit the parameters
    fit_dist = dist(*params)
    distributions[name] = fit_dist
    plt.plot(xx, fit_dist.pdf(xx), label=name)

plt.xlim(xmin, xmax)
plt.ylim(ymin, ymax)
plt.xlabel("price")
_ = plt.legend()

**Note**: we saved the fit distributions in the `distributions` dictionnary for later reuse.

Intuitively we can "see" that the best fit corresponds to the inverse gamma distribution, but we need an objective measure of the quality of the fit.

In the case of one-dimensional distributions, the **goodness of fit** can be measured by the **Kolmogorov-Smirnov** (KS) statistics.

The KS statistic compares an empirical cumulative density function (ECDF) $\hat F(.)$ with an analytical cumulative density function (CDF) $F(.)$.

The CDF is defined as:
$$F(x) = \textrm{Pr}(X \le x)$$

The ECDF $\hat F(x)$ is computed as the proportion of points which are below a given value $x$:

$$\hat F(x) = \frac{1}{N} \sum_{i=1}^N  \mathbb{I}(x_i \leq x)$$

where $\mathbb I(.)$ is the indicator function.

The KS statistic is the maximum difference between the ECDF and the CDF.

$$KS = \max_i{|F(x_i) - \hat F(x_i)|}$$

Let's plot the empirical and the inverse-gamma CDFs and compute the KS statistics.


In [ ]:
#%matplotlib notebook
%matplotlib inline

# We first sort the prices
xs = np.sort(x)

# We just associate to each point the proportion of all preceding points:
ecdf = np.arange(1, len(x) + 1) / len(x)

cdf = distributions["invgamma"].cdf(xs)

plt.plot(xs, ecdf, label="ecdf")
plt.plot(xs, cdf, label="cdf")
plt.legend()

print(f"KS statistic: {np.abs(cdf - ecdf).max():0.2e}")

We can also use the `scipy.stats.kstest` function. Let's do that for three candidate distributions:

In [ ]:
for name, dist in distributions.items():
    print(f"{name:10}:", stats.kstest(x, dist.cdf))

We can verify that the inverse-gamma gives the best (lowest) KS statistic.

Note that the computed $p$-value indicates that it is not very likely that the sample was indeed generated by the fitted distributions but it is usually the case with a high number of real life samples.

Usually, only artificially generated data can reach $p$-values compatible with the null-hypothesis (samples are generated from the proposed distribution):

In [ ]:
stats.kstest(distributions["invgamma"].rvs(size=len(x)), distributions["invgamma"].cdf)

#### Non-parametric density estimation with KDE

Kernel density estimation (KDE) is often used to represent a smoothed version of the
distribution. Most plotting tools actually offer this possibility:

In [ ]:
%matplotlib inline
sns.histplot(x="price", data=df_cars, stat="density", bins=100)
_ = sns.kdeplot(x="price", data=df_cars, color="tab:orange", lw=4, alpha=0.7)

**Note**: We use the `stat="density"` parameter. Among the possible values we have:

* "count": the default
* "frequency": the bin values sum to one.
* "density": the bin values x the bin widths sum to one.

The approximate PDF of the KDE is built by associating a **Gaussian component to each data point**.

Here is an example based on 5 points sampled from the price column.
You can **ignore the code**, and just look at the generated figure:

In [ ]:
xs = df_cars.sample(5, random_state=4).price.values

# Compute the data bounds and range
xmin, xmax = xs.min(), xs.max()
delta = xmax - xmin

# We plot the location of samples along the x axis
plt.scatter(xs, np.zeros_like(xs), marker="+", s=200)
xx = np.linspace(xmin - 1.2 * delta, xmax + 1.2 * delta, 501)

# Set the width of the Gaussians
sigma = 5000
pdfs = []

for x in xs:
    # We compute the fraction of the pdf associated to the current point
    # The Gaussien components is centered on x and has with sigma
    x_pdf = 1 / len(xs) * stats.norm.pdf(xx, loc=x, scale=sigma)

    # We plot the pdf of each Gaussian
    plt.plot(xx, x_pdf, color="lightgray", linestyle="--")

    # We plot the center of the gaussian
    plt.plot([x, x], [0, x_pdf.max()], color="lightgray", linestyle="--")
    pdfs.append(x_pdf)

# We plot the complete PDF by summing the individual PDFs
plt.plot(xx, np.sum(pdfs, 0))
plt.xlabel("price")
_ = plt.ylabel("pdf")

The width of the individual Gaussian components, also called the **bandwidth** is given here as a multiplicative factor of the sample standard deviation $\hat \sigma$.

This parameter is important since too small values will lead to mis-represent inter-sample PDF, while too big values will smooth out the PDF structure. Moreover, we intuitively see that the more sampling points we have, the smaller the bandwidth has to be.

The bandwidth is usually chosen using some heuristics like Scott's rule:

$$\sigma_{Scott} = \hat \sigma \times N^{-\frac{1}{d + 4}}$$

where $\hat \sigma$ is the empirical standard deviation, $d$ is the number of dimensions of the dataset (here 1) and $N$ the total number of points.

In [ ]:
x = np.arange(1, 1000)
plt.plot(x, x ** (-1 / 5))
plt.xlabel("Number of sampling points")
_ = plt.ylabel("Scott's factor")

The `scipy.stats.gaussian_kde` function allows us to easily compute this approximation. The `bandwidth` of the Gaussian KDE can be set as a factor of the empirical standard deviation $\hat \sigma$, through the `bw_method` parameter. By default, it uses Scott's rule.

Let's see how it behaves:

In [ ]:
from scipy.stats import gaussian_kde

plt.scatter(xs, np.zeros_like(xs), marker="+", s=200)
for bw in [1.0, 0.5, 0.3, 0.1]:
    kde = gaussian_kde(xs, bw_method=bw)
    plt.plot(xx, kde.pdf(xx), "--", label=f"kde factor: {kde.factor:0.2f}")

# Using the default "Scott's rule"
kde = gaussian_kde(xs)
plt.plot(xx, kde.pdf(xx), label=f"Scott's rule: {kde.factor:0.2f}", lw=4, alpha=0.5)
_ = plt.legend()

**Note**: KDE must be used with caution with bounded variable like our price variable, since the Gaussian kernel is unbounded. In the example above the KDE PDF is not null for negative values, while negative prices are impossible.

The object built by the `gaussian_kde` function (it is actually a constructor) is callable. Using `kde(x)` is actually equivalent to using `kde.pdf(x)`. Other interesting methods are:

* `logpdf()`: Useful to compute log likelihood of a new sample w.r.t the estimated pdf.
* `resample()`: Allows us to sample new points from the the estimated pdf.

As already stated, most plotting tools provide ways to plot KDE. The following example use Pandas and Seaborn respectively:

In [ ]:
df_cars.price.plot.kde()
plt.xlim(-1500, 170000)
_ = plt.xlabel("price")

In [ ]:
_ = sns.kdeplot(data=df_cars, x="price")

Plotting KDE can be more convenient than histograms to compare several distributions on the same graph:

In [ ]:
_ = sns.kdeplot(data=df_cars, x="price", hue="transmission", common_norm=False)

### Multivariate analysis

It can be interesting to understand the way different variables interact with each other, or with the target.



#### Scatter plots

Scatter plots allow us to visualize the interaction between 2 continuous variables. Additional information (continuous or categorical) can be added using the marker color, size or shape.

However, when the number of samples is high, it becomes difficult to analyze:

In [ ]:
quality_columns = [c for c in df_cars_prep.columns if c.endswith("_quality")]
mask_ok = (df_cars_prep[quality_columns] == "ok").all(1) & ~df_cars_prep.tax_mpg_missing
df_cars_clean = df_cars_prep[mask_ok].drop(
    quality_columns + ["tax_mpg_missing"], axis=1
)

In [ ]:
_ = sns.scatterplot(
    data=df_cars_clean.sample(frac=0.01, random_state=1),
    x="mileage",
    y="price",
    hue="year",
    palette="viridis",
)

In the above example we can see a gradient of the `year` variable when the `mileage` increases.

#### Histograms
When the number of points is too high, 2d histograms can be used instead of scatter plots. The complete dataset can then be used. The color represents the density:

In [ ]:
_ = sns.histplot(
    data=df_cars_clean, x="mileage", y="price", bins=100, cmap="plasma", cbar=True
)

Or using directly matplotlib with color log-scale:

In [ ]:
from matplotlib.colors import LogNorm

plt.hist2d(
    df_cars_clean.mileage, df_cars_clean.price, bins=100, cmap="plasma", norm=LogNorm()
)
_ = plt.colorbar()

#### Kde plots

Kde plots are another way to represent the density of the point cloud. The computation time can increase significantly with the number of points (computation complexity $O(n^2)$), so it is recommended to subsample the dataset if the number of data points is high. However, it has the advantage to allow representing an additional categorical information:

In [ ]:
_ = sns.kdeplot(
    data=df_cars_clean.sample(frac=0.01, random_state=1),
    x="mileage",
    y="price",
    hue="transmission",
)

The contours represent the iso-density curves, with the inner-most curves corresponding to the highest densities.

#### Box plots

Box plots allow to summarize the distribution of 1 continuous variable as a function of at most 2 categorical variables:

In [ ]:
plt.figure(figsize=(12, 5))
sns.boxplot(data=df_cars_clean, x="brand", y="price", hue="fuelType")
_ = plt.ylim(0, 80000)

##### Violin plots

Violin plots show the full distribution for each categorical value using KDE:

In [ ]:
plt.figure(figsize=(12, 5))
sns.violinplot(data=df_cars_clean, x="brand", y="price", hue="fuelType")
_ = plt.ylim(0, 80000)

### Dimension reduction

**Dimension reduction** algorithm allows us to sum up the information contained in a high-dimensional data set into fewer variables. The aim is thus to capture lower dimensional structures in high-dimensional space. For this reason, dimension reduction techniques can be considered as **manifold learning** algorithms.

In the figure below we show some examples of 2d manifolds in a 3d space. We have from left to right:

* a linear manifold
* a non linear manifold
* a closed sphere

While the 2 first cases can in theory easily be unfolded onto a 2d surface, closed structures like the sphere will generally not be easy to project.

In [ ]:
grid = np.linspace(-1, 1, 21)
xx, yy = np.meshgrid(grid, grid)

plt.figure(figsize=(15, 8))
ax = plt.subplot(1, 3, 1, projection="3d")
zz = xx + 2.0 * yy + 3
ax.plot_wireframe(xx, yy, zz)

ax = plt.subplot(1, 3, 2, projection="3d")
zz = 3 * xx**2 + 2.0 * yy**3 + 3
ax.plot_wireframe(xx, yy, zz)


pp, tt = np.meshgrid(np.linspace(0, 2 * np.pi, 21), np.linspace(0, np.pi, 11))
xx = np.cos(pp) * np.sin(tt)
yy = np.sin(pp) * np.sin(tt)
zz = np.cos(tt)
ax = plt.subplot(1, 3, 3, projection="3d")
_ = ax.plot_wireframe(xx, yy, zz)

#### PCA

The simplest lower-dimensional structure in a high dimensional space corresponds to a **linear subspace**.

Principal Component Analysis finds the directions of **maximal variance**. It builds a set of orthogonal directions, with an associated variance along each one. Those direction are usually sorted by decreasing value of their variance. For dimension reduction we keep only the axes corresponding to the highest variance values.

Let's build a toy dataset with 2 steps:

* build a 2d spiral shaped dataset
* add noise in 3d
* perform a different scaling for each axis, a rotation in 3d and a translation

In [ ]:
np.random.seed(1)

t = np.linspace(0, 1, 101)
a = 10.0
x = t**0.6 * np.cos(a * t)
y = t**0.6 * np.sin(a * t)
z = np.zeros_like(t)

X_ = np.vstack([x, y, z]).T
X_ += 0.05 * np.random.randn(*X_.shape)
X_ /= X_.std(0)


u = np.array(
    [
        [1, 1, -2],
        [1, -1, 0],
        [1, 1, 1],
    ]
).T
u / (u**2).sum(0)
w = np.array([4, 1, 0.25])
mu = [-1, 1, 2]
X_ = mu + (X_ * w) @ u

We can now plot it:

In [ ]:
elevs = [10, 30, 70]
azims = [10, 20, 70]

fig = plt.figure(figsize=(12, 9))
for a, (elev, azim) in enumerate(zip(elevs, azims)):
    ax = fig.add_subplot(1, 3, a + 1, projection="3d")
    ax.scatter(*X_.T, alpha=0.1)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")
    ax.view_init(elev=elev, azim=azim)
plt.tight_layout()

We can clearly see the dataset structure. The main directions of the data set are show below:

In [ ]:
fig = plt.figure(figsize=(12, 9))
for a, (elev, azim) in enumerate(zip(elevs, azims)):
    ax = fig.add_subplot(1, 3, a + 1, projection="3d")
    ax.scatter(*X_.T, alpha=0.1)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")
    ax.view_init(elev=elev, azim=azim)

    for i in range(3):
        ax.plot(
            [mu[0], mu[0] + u[i, 0] * np.sqrt(w[i]) * 3],
            [mu[1], mu[1] + u[i, 1] * np.sqrt(w[i]) * 3],
            [mu[2], mu[2] + u[i, 2] * np.sqrt(w[i]) * 3],
            lw=4,
        )
plt.tight_layout()

Those main directions correspond more or less to the directions along which the variance is maximal.

The Principal Component Algorithm aims at finding the an orthonormal basis aligned with the directions of maximal variance.

##### Covariance and correlation

We have here $N$ points in 2 dimensions. The sample covariance between the two variables $x$ and $y$ is measured as :

$$\textrm{Cov}(x, y) = \frac{1}{N-1}\sum_{i=1}^N (x_i - \bar x)(y_i - \bar y)$$

The correlation is a normalized version of the covariance:


$$\textrm{Corr}(x, y) = \frac{\textrm{Cov}(x, y)}{\sqrt{\textrm{Var}(x)\textrm{Var}(y)}}$$

Note that if all variables have unit variance the covariances and correlations are the same.

The variance-covariance matrix or for short the covariance matrix is the matrix that contains all pairwise (co-)variances:

In [ ]:
C = np.cov(X_.T)
C

The diagonal of $C$ corresponds to variance of the variables and the off-diagonal elements to the covariances.

In [ ]:
X_.var(0)

**Exercise**: The values on the diagonal and the computed variances are close but not exactly the same. Can you explain why ?

The computation of the variance may use $\frac{1}{N}$ instead of $\frac{1}{N-1}$. The $N-1$ should be used when the mean is not known in advance.

In [ ]:
X_.var(0, ddof=1)

You can compute the correlation matrix using:

In [ ]:
np.corrcoef(X_.T)

The correlation between x and y is given by the off-diagonal element.

##### Finding the principal components

The principal components are retrieved by computing the eigen-decomposition of the covariance matrix:

$$C \boldsymbol u_i = \lambda_i \boldsymbol u_i$$

The eigen-vector $\boldsymbol u_i$ correspond to the principal components while the eigen-values $\lambda_i$ correspond to the variance along the corresponding component.

The principal components are usually sorted according to the decreasing variance values.

##### Properties of the projected data points


The input data are then projected on the principal axis, or equivalently the data points are rotated so that they align with the principal components.

As a consequence, **in the projected space, the covariance matrix is diagonal** or otherly said **in the projected space, the variables are uncorrelated**.

We perform **dimensional reduction** by keeping only the **first few principal components associated with the highest variances**.


The overall variance of the dataset is often measured as the sum of the variance along each axis and we have this interesting property:

$$ V_{tot} = \sum_{j=1}^d C_{jj} = \sum_{k=1}^d \lambda_k$$


where $\lambda_k$ is the variance along the $k$-th principal component while $C_{jj}$ is the variance along the $j$-th variable in the input space. (The equality comes from the property that the trace of a symmetric matrix is invariant by rotation).

So the **explained variance** associated to each principal component is often measured as the ratio:

$$\rho_k = \frac{\lambda_k}{V_{tot}}$$

And the **cumulated explained variance** preserved by projecting the data points on the first $K$ principal components is measured as:

$$\sum_{k=1}^K \rho_k$$

##### Transformers

The scikit-learn toolkit is based on various types of **estimators**. Some of those estimators are dedicated to data transformations, the so called **transformers** estimators.

Transformer classes derive from the `TransformerMixin` base class. Transformer classes implement at least the following methods:

* `fit(X, y=None)`: to train the transformer.
* `transform(X)`: to apply the transformation.
* `fit_transform(X, y=None)`: chain the training and application stages

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
X_tr_ = pca.fit_transform(X_)

The `PCA` object, once fit, has the following attributes:

* `components_`: holds the principal components
* `explained_variance_` which holds the corresponding variances
* `explained_variance_ratio_` which holds the explained variance divided by the total variance.

In [ ]:
pca.components_

In [ ]:
pca.explained_variance_

In [ ]:
pca.explained_variance_ratio_

In [ ]:
plt.figure(figsize=(12, 8))
plt.subplot(2, 2, 1)
plt.plot(pca.explained_variance_ratio_, "o-")
plt.xticks([0, 1, 2], [1, 2, 3])
plt.xlabel("Principal Component")
plt.ylabel("Explained variance ratio")
plt.subplot(2, 2, 2)
plt.plot(pca.explained_variance_ratio_.cumsum(), "o-")
plt.xticks([0, 1, 2], [1, 2, 3])
plt.xlabel("Principal Component")
plt.ylabel("Cumulated explained variance ratio")

plt.subplot(2, 1, 2)
plt.scatter(X_tr_.T[0], X_tr_.T[1])
plt.xlabel("PC1")
_ = plt.ylabel("PC2")

In the figure above we show:

* The explained variance ratio for each PC
* The explained variance ration cumulated over PC
* The projection over the two first principal components.

The analysis of the 2 first graphs shows that most of the information is carried by the first 2 PCs.

The 3rd graphs show the data points once projected on those 2 PCs.

It can be interesting to give the projected dataset unit variance along each axis. This is called spherization or whitening:

In [ ]:
from sklearn.decomposition import PCA

plt.figure(figsize=(5, 5))
pca = PCA(2, whiten=True)
Xtr = pca.fit_transform(X_)

plt.scatter(*Xtr.T)
xmin, xmax = plt.xlim()
plt.ylim(xmin, xmax)
plt.xlabel("pc1")
_ = plt.ylabel("pc2")

##### Applying PCA

Let's see how to use this tool on our dataset. We can use it on the **numerical values only**.

A first step usually consists in **standardizing** the input variables. This consists in:

* Removing the mean
* Dividing by the standard deviation.

This is mainly done for removing scale effects due to the units used to express the input variables. By giving them the same variance, they are given equal importance in the final result.

The `sklearn.preprocessing.StandardScaler` transformer allows us to perform standardization:

In [ ]:
from sklearn.preprocessing import StandardScaler

num_cols = [c for c in df_cars_clean.columns if df_cars_clean.dtypes[c] != "object"]
print("Numerical columns:", num_cols)
X_num = df_cars_clean[num_cols]

X_scaled = StandardScaler().fit_transform(X_num)

We can then apply PCA. In the example below we keep all components to illustrate how we can analyze the **explained variance**:

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=len(num_cols), whiten=True)
X_proj = pca.fit_transform(X_scaled)

We can then examine the explained variance ratios:

In [ ]:
plt.plot(pca.explained_variance_ratio_, "o-")
plt.grid("on")
ymin, ymax = plt.ylim()
plt.ylim(0, ymax)
plt.xlabel("Axis number")
_ = plt.ylabel("Explained variance ratio")

Examining the cumulated variances can give an idea of the amount of information lost when projecting on a lower dimension:

In [ ]:
plt.plot(pca.explained_variance_ratio_.cumsum(), "o-")
plt.grid("on")
ymin, ymax = plt.ylim()
plt.ylim(0, ymax)
ticks = np.arange(pca.n_components)
plt.xticks(ticks, ticks + 1)
plt.xlabel("Number of axes kept")
plt.ylabel("Cumulated explained variance ratio")

The graphic above indicates that projecting on the first 3 axes allows to keep around 85% of the total variance.

Let's add those projection components to the data frame:

In [ ]:
df_cars_pca = df_cars_clean.assign(pc1=X_proj.T[0], pc2=X_proj.T[1], pc3=X_proj.T[2])

We can now visualize the projections:

In [ ]:
from matplotlib.colors import LogNorm

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.histplot(data=df_cars_pca, x="pc1", y="pc2", cmap="viridis")
plt.subplot(1, 2, 2)
_ = sns.histplot(data=df_cars_pca, x="pc1", y="pc3", cmap="viridis")

It's important to keep in mind that PCA performs some projections on orthogonal axes. This means that:

* points which are far away in the projected space are also far away in the input space
* points which are close in the projected space are not necessarily close in the input space (they can be far away on an orthogonal axis)

However, since principal components are sorted according to decreasing values of variances, the last axes are supposed to carry less information.

##### Correlation circles

Principal components are expressed in the space of explanatory variables. However, to better understand the meaning of the projected point cloud, it can be interesting to visualize something like the projection of the features in the projected coordinates.

To do this, we usually plot the linear correlation between the projection axes and the original features. Assuming the input features have unit variance, the correlation between principal component $i$ and the feature $j$ is:

$$ c_{ij} = u_{ij}\sqrt{\lambda_i} $$

Where $u_{ij}$ is the $j$-th component of the $i$-th principal component and $\lambda_i$ its associated variance.

In [ ]:
from mll1.plotting import plot_correlation_circle

plt.figure(figsize=(12, 12))

plt.subplot(2, 2, 1)
sns.histplot(data=df_cars_pca, x="pc1", y="pc2", cmap="viridis")
plt.subplot(2, 2, 2)
sns.histplot(data=df_cars_pca, x="pc1", y="pc3", cmap="viridis")

plt.subplot(2, 2, 3)
plot_correlation_circle(pca, [0, 1], num_cols)
plt.subplot(2, 2, 4)
plot_correlation_circle(pca, [0, 2], num_cols)

The lower graphs allow to analyze two things:

* the direction gives an indication on the way the variable values evolve in the principal planes above
* the angle between the variable indicate the way variables are linked in the principal plane

For instance:

* mileage and year are strongly anti-correlated in both planes
* price and tax are strongly correlated in the first plane while almost non correlated in the second


So we can guess that the small cluster at the right of both plots most probably corresponds to a subset of points with very high values of mileage and mpg.

Ideally we would want to select subparts of the graphs and see what the distributions of the different variables are. While this can be done manually, it is more confortable to perform this kind of analysis interactively.

#### t-SNE

The t-SNE algorithm for (t-distributed stochastic neighbor embedding), tries to put in correspondence two probability distribution functions, one computed in the high-dimensional input space and one in the lower dimensional output space.

The algorithm models the joint probability of observing two points $i$ and $j$ based on the their distances $d_{ij}$ in the input space and $d'_{ij}$ in the output space.

Roughly speaking the joint probability in the input space is based on a Gaussian distribution (thin tail) for the input space and a t-Student distribution (fat tail) in the output space. The two joint probabilities are put in correspondence by minimizing their Kullback-Leibler divergence.

$$D(p || q) = \sum_i \sum_{j \neq i} p_{ij} \log\frac{p_{ij}}{q_{ij}}$$

$$q_{ij} = \frac{(1 + \|\boldsymbol y_i - \boldsymbol y_j\|^2)^{-1}}{\sum_{k \neq i}(1 + \|\boldsymbol y_i - \boldsymbol y_k\|^2)^{-1}}$$

$$p_{ij} = \frac{p_{i|j} + p_{j|i}}{2}$$

$$p_{i|j} = \frac{\exp(-\|x_i - x_j\|^2 / 2\sigma_i^2)}{\sum_{k \neq i}\exp(-\|x_i - x_k\|^2 / 2\sigma_i^2}$$


being the Shanon entropy. The algorithm performs a binary search for the value of $\sigma_i$ giving a fixed value of the perplexity given by the user.

Let $P_i$ be the conditional probability of $x_i$ conditionned by all other points, the perplexity is defined by:

$$Perp(P_i) = 2^{H(P_i)}$$

With

$$H(P_i) = -\sum_j p_{i|j} \log_2 p_{i|j}$$

being the entropy of $P_i$.

Note that the algorithm needs to evaluate the distance between all pairs of points. This means that it is computationally in $O(N^2)$ so, for big datasets we might need to subsample:

In [ ]:
from sklearn.manifold import TSNE
from sklearn.utils import check_random_state

rng = check_random_state(1)

df_cars_sample = df_cars.dropna().sample(n=1000, random_state=rng)

X_num = df_cars_sample[num_cols].values
X_scaled = StandardScaler().fit_transform(X_num)

tsne = TSNE(n_components=2, random_state=rng)

X_proj = tsne.fit_transform(X_scaled)

df_cars_tsne = df_cars_sample.assign(
    tsne1=X_proj.T[0],
    tsne2=X_proj.T[1],
)

_ = sns.scatterplot(data=df_cars_tsne, x="tsne1", y="tsne2", hue="fuelType")

The main drawback of this algorithm is the impossibility to project new points. As a consequence this algorithm should be used for data exploration and visualization only and **it should not be used as a preprocessing step**.

## Going further

### Other algorithms

Refer to the documentation of the [`sklearn.manifold`](https://scikit-learn.org/stable/modules/manifold.html) sub-package for a list of manifold learning algorithms.


#### Distance based algorithm

The t-SNE algorithm can be seen as a distance based algorithm where distances in the input space are matched to the output distances.

The most representative algorithm of this family is the **Multi-dimensional scaling** algorithm were the difference between pairwise distances is directly minimized.

#### Graph based algorithms

Graph based algorithms use the **nearest neighborhood graph** to capture the local structure of the dataset. They then use this structure to try to embed the data points in a lower dimensional space.

Among the most common algorithms one can site:

* Isomap: aims at reproducing the Geodesic distance on the graph (linked to MDS)
* Spectral Embedding: Uses the eigen decomposition of the graph Laplacian. This can be understood as an approximation of the commute time distance on the graph. See for instance [this paper](https://www.academia.edu/download/2136944/5zoa93jb40vzxxq.pdf).
* Locally Linear Embedding: tries to express each point as a linear combination of its neighbors

In the example below, we show an example which uses the **locally linear embedding**. It has the main advantage to be able to project new points:

In [ ]:
from sklearn.manifold import LocallyLinearEmbedding

rng = check_random_state(1)

X_num = df_cars.dropna()[num_cols].values
X_scaled = StandardScaler().fit_transform(X_num)

lle = LocallyLinearEmbedding(n_components=2, n_neighbors=30, random_state=rng)
isub = rng.permutation(len(X_scaled))[:1000]
lle.fit(X_scaled[isub])

X_proj = lle.transform(X_scaled)

plt.figure(figsize=(8, 8))
_ = sns.histplot(x=X_proj.T[0], y=X_proj.T[1], cmap="viridis")

#### Auto-encoders

Auto-encoders are multi-layer neural networks that try to reproduce their inputs while imposing a low-dimensional inner representation.

They can be understood as two networks:

* The encoder is used to project the data points in the low dimensional space;
* The decoder is used to project back the data points in the original space.

The network is trained by minimizing the distance between the input and the output.

We show here a dirty **hack** allowing to use the multi-layer perceptron implementation of `sklearn`. A MLP is a network composed of successive linear projections with an non-linear function applied element-wise between each projection step.

The output of layer $l$ is typically:

$$ X_l = a(X_{l - 1}W_l) $$

Where $X_{l-1}$ is the output if the previous layer and $W_l$ is the projection matrix of the $l$-th layer of size $D_{l-1} \times D_{l}$.

A MLP applies those steps iteratively starting from the input data:

$$X_{out} = a(\dots a(a(X_{in} W_1)W_2) \dots W_{L-1})W_L$$

Note that the last step usually does not use the activation function so that it can match the scale of the values to predict.

For an auto-encoder we split the network into two sub-networks:

$$X_{enc} = \mathrm{enc}(X_{in})= a(\dots a(a(X_{in} W_1)W_2) \dots W_{L/2-1})W_{L/2}$$

$$X_{out} = \mathrm{dec}(X_{enc}) = a(\dots a(a(X_{enc}) W_{L/2 + 1}) \dots W_{L-1})W_{L} $$

We first train a regression network which reproduces its input as output:

In [ ]:
from sklearn.neural_network import MLPRegressor

mlp = MLPRegressor(hidden_layer_sizes=(10, 2, 10), max_iter=200, random_state=1)
mlp.fit(X_scaled, X_scaled)

Note the hidden layer size 2, which allows to impose the projection in 2 dimensions. The network will be composed of 4 layers:

In [ ]:
for layer in mlp.coefs_:
    print(layer.shape)

We can compute the mean distance between the input and the output:

In [ ]:
X_out = mlp.predict(X_scaled)
np.mean((X_scaled - X_out) ** 2)

We now use a hack to build a new network with only the first layers:

In [ ]:
from copy import deepcopy

In [ ]:
mlp_proj = deepcopy(mlp)
mlp_proj.hidden_layer_sizes = mlp_proj.hidden_layer_sizes[:2]
mlp_proj.n_layers_ = len(mlp_proj.hidden_layer_sizes)
mlp_proj.n_outputs_ = 2
mlp_proj.coefs_ = mlp.coefs_[:2]

We can now project points using the predict method (we will explain the meaning of this method later):

In [ ]:
X_proj = mlp_proj.predict(X_scaled)
plt.figure(figsize=(8, 8))
_ = sns.histplot(x=X_proj.T[0], y=X_proj.T[1], cmap="viridis")

Note that PCA can be seen as a 2-layers auto-encoder with no activation function.

The main drawbacks of auto-encoders are:

- their sensitivity to the initialization
- their training times
- the difficulty to interpret the output (as for all non-linear projections)

### Interactive analysis with Altair*

Altair is a plotting tool for Python based on the Vega-lite JavaScript tool. It makes it relatively easy to build interactive analyses, but the processing is actually performed by the JavaScript code, and the full dataset needs to be duplicated in memory.

To prevent memory and computation overload, we will need to subsample.

We first declare the main `Chart` object holding the dataset:

In [ ]:
import altair as alt

alt.data_transformers.disable_max_rows()

base = alt.Chart(
    # We sub-sample so that the result it not too slow
    # Choose the random_state carefully
    df_cars_pca.sample(frac=0.05, random_state=4)
)

Then we declare a selection object that will be used to draw box selections in the scatter plots and filter the result in the other plots:

In [ ]:
# We declare a selection object
# It will be used to have a box selection in the scatter plots
# And dynamically recompute other graphs
select = alt.selection_interval(name="2D_select")

We will plot two scatter plots with different principal components. Since they will be very similar we write a function to factorize the code:

In [ ]:
def scatter(col1, col2):
    return (
        base.mark_point()
        .encode(
            x=f"{col1}:Q",
            y=f"{col2}:Q",
            color=alt.condition(select, "brand:N", alt.value("lightgray")),
            opacity=alt.condition(select, alt.value(0.7), alt.value(0.1)),
            tooltip=["brand:N", "model:N"],
        )
        .add_selection(select)
        .properties(width=300, height=150)
    )

Then we write a function to plot histograms for the various variables. Continuous variables will need to be binned. To keep the histogram consistent for all selections, we first plot the full histogram in gray and overlap the selected histogram on top:

In [ ]:
def hist_with_background(column, bins=None):
    xcol = (
        f"{column}:N" if bins is None else alt.X(f"{column}:Q", bin={"maxbins": bins})
    )

    hist_base = base.mark_bar().encode(
        x=xcol, y=alt.Y("count()", scale={"type": "log"})
    )

    return (
        hist_base.encode(color=alt.value("lightgray"))
        + hist_base.transform_filter(select)
    ).properties(width=200, height=150)

Now we can declare the various graph components:

In [ ]:
scatter_pca12 = scatter("pc1", "pc2")
scatter_pca13 = scatter("pc1", "pc3")

hist_price = hist_with_background("price", bins=50)
hist_mileage = hist_with_background("mileage", bins=50)
bar_brand = hist_with_background("brand").encode(color="brand:N")

hist_mpg = hist_with_background("mpg", bins=50)
bar_year = hist_with_background("year")
bar_fuel = hist_with_background("fuelType")

Finally we organize the final graph component:

In [ ]:
# The layout
graph = (
    (scatter_pca12 | scatter_pca13)
    & (hist_price | hist_mileage | bar_brand)
    & (hist_mpg | bar_year | bar_fuel)
)

And plot the result:

In [ ]:
graph

### PCA

#### Derivation

PCA looks for the unit projection vector $\boldsymbol u$ which maximizes the variance:

$$ \max_{\boldsymbol u} Var(X \boldsymbol u) $$
$$ \text{s.t.} \|\boldsymbol u\|^2 = 1 $$

Assuming for simplicity that the point cloud X has zero mean:

$$Var(X \boldsymbol u) = \frac{1}{N - 1}\sum_{i=1}^N (\boldsymbol u^T X^T)(X \boldsymbol u)$$

$$Var(X \boldsymbol u) = \boldsymbol u^T \left(\frac{1}{N - 1}\sum_{i=1}^N X^T X\right) \boldsymbol  u$$

$$Var(X \boldsymbol u) = \boldsymbol u^T C \boldsymbol  u$$

So must solve:

$$ \max_{\boldsymbol u} \boldsymbol u^T C \boldsymbol  u $$
$$ \text{s.t.  } \|\boldsymbol u\|^2 = 1 $$

Using Lagrange multipliers this is equivalent to solve:


$$\nabla_{\boldsymbol u} (\boldsymbol u^T C \boldsymbol u) = \nabla_{\boldsymbol u} (\boldsymbol u^T \boldsymbol u)$$

Which means that the gradient is perpendicular to the constraint. This leads to:

$$ 2 C \boldsymbol u = 2 \lambda \boldsymbol u $$

Which is an eigen-decomposition problem. There are several $\lambda$ eigenvalues and corresponding $\boldsymbol u$ eigenvectors:

$$ C \boldsymbol u_i = \lambda_i \boldsymbol u_i $$

The maximum eigenvalue $\lambda_i$ corresponds to the axis of maximum variance. This eigenvalue decomposition allows to build an orthonormal basis which can be seen as a rotation matrix.

#### Diagonalization of the Covariance matrix

In the rotated space, the covariance matrix is the diagonal matrix $\Lambda$ with the $\lambda_i$ values on the diagonal:

$$ C = U\Lambda U^T$$


$$\Lambda = \left(\matrix{
\lambda_1 & 0         & \dots  & 0         \\
0         & \lambda_2 & \dots  & 0         \\
\vdots    & \vdots    & \ddots & \vdots    \\
0         & 0         & \dots  & \lambda_d \\
}\right)$$

Where the $i$-th column of the $U$ matrix corresponds to the eigenvector / principal component $\boldsymbol u_i$.

The diagonal shape of the covariance matrix implies that **in the projected space, the variables are uncorrelated.**

#### Dimension reduction and noise filtering

It is possible to use the principal components to project points in the low dimensional space back in the original space.

Remind that the projection is performed using:

$$ X' = XU $$

Where $U$ is the $d \times K$ matrix where the $K$ columns corresponding to the $K$ first principal components.

We can then use $U^T$ to project back in the original space:

$$ X'' = X'U^T = XUU^T $$

By imposing the projection into the lower dimensional space, the components along the lowest variance components are removed. This acts like a noise filter.

In the example below, we build a dataset made of points aligned on the xy plane and add noise along the z-axis:

In [ ]:
grid = np.linspace(-1, 1, 21)
xx, yy = np.meshgrid(grid, grid)
zz = np.sqrt(0.01) * np.random.randn(*xx.shape)

plt.figure(figsize=(8, 8))
ax = plt.axes(projection="3d")
ax.scatter(xx, yy, zz, s=100)
_ = ax.set_zlim(-1, 1)

In [ ]:
X_ = np.dstack([xx, yy, zz]).reshape(-1, 3)

pca = PCA(n_components=2)
X_proj = pca.fit_transform(X_)

plt.figure()
plt.scatter(*X_proj.T)

Note that the 2 first eigenvalues are equal in theory and all rotations in the xy plane lead to equivalent solutions:

In [ ]:
pca.explained_variance_ratio_

In [ ]:
X_back = pca.inverse_transform(X_proj)

plt.figure(figsize=(8, 8))
ax = plt.axes(projection="3d")
ax.scatter(*X_back.T, s=100)
ax.set_zlim(-1, 1)

We see that the noise along the z-axis as been removed. When using PCA to filter low variance noise, one can understand PCA as a **linear auto-encoder**.

#### Correlation circles

##### Definition
The correlation circles are used to visualize the link between principal components and the original features.

Original features are represented in a plane using the correlation between the considered input variable and the factors associated to 2 principal components. If $c_{i_1 j}$ and $c_{i_2 j}$ correspond to the correlations between the original variable $j$ and the factors associated to the principal components $i_1$ and $i_2$ then the features $j$ is represented as the $(c_{i_1 j}, c_{i_2 j})$ point.

The name *correlation circle* is due to the fact that, by construction:

$$c_{i_1 j}^2 + c_{i_2 j}^2 \leq 1$$

since principal components are orthogonal and the **points lie inside the unit circle**.

##### Derivation
We want to compute the correlation between the PCA factors and the original variables.

Let's first compute the corresponding covariance. We assume points are centered (0 mean) for simplicity:

$$ Cov(X, X) =  C = \frac{1}{N - 1} X^T X = U\Lambda U^T $$

$$ X' = XU $$

$$\begin{eqnarray}
\mathrm{Cov}(X', X) & = & \frac{1}{N - 1} X'^T X \\
                    & = & \frac{1}{N - 1} U^T X^T X \\
                    & = & U^TC \\
                    & = & U^T U\Lambda U^T
\end{eqnarray}
$$

By construction $$U^T U = I_K$$ where $I_K$ is the $K\times K$ identity matrix. Then:

$$ \mathrm{Cov}(X', X) = \Lambda U^T $$

So the correlation matrix can be written as:

$$
\begin{eqnarray}
\mathrm{Corr}(X', X) & = & \Lambda^{-1/2} \mathrm{Cov}(X', X) \mathrm{Diag}(C)^{-1/2} \\
                     & = & \Lambda^{1/2} U^T \mathrm{Diag}(C)^{-1/2}
\end{eqnarray}$$

So if $\boldsymbol u_i$ is the $i$-th principal component and $u_{ij}$ the $j$-th component of the $i$-th principal component. The correlation $c_{ij}$ between the $i$-th factor (projection of the dataset on the $i$-th principal component) and the $j$-th input variable is:

$$ c_{ij} = \frac{u_{ij} \sqrt{\lambda_i}}{\sqrt{C_{jj}}} $$

### Density estimation and variable change

As we have seen earlier, one of the main problems with KDE relies in the fact that the Gaussian kernel can lead to non-null PDF values on the negative domain.

One way to solve the problem of the definition domain of the KDE is to apply it on a transformed version of the input variable and convert the obtained PDF so that it applies to the original domain.

This however requires to treat the variable change correctly:

$$ |p_x(x) dx| = |p_y(y) dy| $$

$$y = f(x) $$

$$ p_x(x) = p_y(f(x)) \left| \frac{d f(x)}{dx}\right| $$

We show below how to perform KDE using the `scipy.stats.gaussian_kde()` function for a 1D variable:

In [ ]:
price = df_cars_prep.price.values
_ = plt.hist(price, bins=100, density=True)

If we look at the distribution of the logarithm instead we obtain:

In [ ]:
_ = plt.hist(np.log(price), bins=100, density=True)

In [ ]:
from scipy.stats import norm

plt.hist(np.log(price), bins=100, density=True)
params = norm.fit(np.log(price))

xmin, xmax = plt.xlim()
xx = np.linspace(xmin, xmax, 101)
_ = plt.plot(xx, norm(*params).pdf(xx))

So to apply the fitted PDF in the original definition domain we use:

$$y = \log(x) $$

$$ p_x(x) = p_y(\log(x)) \left| \frac{d\log(x)}{dx}\right| = \frac{p_y(\log(x))}{x} $$

In [ ]:
plt.hist(price, bins=100, density=True)
xx = np.linspace(1, price.max(), 101)
_ = plt.plot(xx, norm(*params).pdf(np.log(xx)) / xx)

This of course works the same way with KDE

In [ ]:
plt.hist(price, bins=100, density=True)
kde = stats.gaussian_kde(np.log(price))
x = np.linspace(0, 150000, 101)[1:]
_ = plt.plot(x, kde(np.log(x)) / x)

It can be applied in 2d too:

$$p_{x, y}(x, y) = p_{u, v}(u, v) \left|
\matrix{
    \frac{\partial u}{\partial x} & \frac{\partial v}{\partial x} \\
    \frac{\partial u}{\partial y} & \frac{\partial v}{\partial y} \\
}
\right|$$

with

$$u = \log(x)$$
$$v = \log(y)$$

this leads to:

$$p_{x, y}(x, y) = \frac{p_{u, v}(\log(x), \log(y))}{xy}$$

In [ ]:
X_ = df_cars_prep[["mileage", "price"]].sample(5000, random_state=1).values

kde_log = stats.gaussian_kde(np.log(X_.T))
xr = np.linspace(0, X_.T[0].max() * 0.5, 101)[1:]
yr = np.linspace(0, X_.T[1].max() * 0.5, 101)[1:]
xx, yy = np.meshgrid(xr, yr)
X_log = np.log(np.array([xx, yy])).reshape(2, -1)
zz_log = kde_log(X_log).reshape(xx.shape)
plt.contour(xx, yy, zz_log / xx / yy, levels=11)
plt.xlabel("mileage")
_ = plt.ylabel("price")